# **Task 1: Documentation ingestion and vector database creation**

In [ ]:
!pip install -U \
  langchain \
  langchain-community \
  langchain-text-splitters \
  chromadb \
  sentence-transformers \
  pypdf


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 68.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.7/21.7 MB 35.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329.6/329.6 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 44.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 41.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 9.4 MB/s eta 0:00

In [ ]:
# -------------------------------
# STEP 1: Imports
# -------------------------------
import os
import re
from typing import List, Dict

from pypdf import PdfReader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma

In [ ]:
# -------------------------------
# STEP 2: PDF Loader + Cleaning
# -------------------------------
def load_pdf(file_path: str) -> str:
    """Extract text from PDF file."""
    reader = PdfReader(file_path)
    text = ""
    for page in reader.pages:
        page_text = page.extract_text()
        if page_text:
            text += page_text + "\n"
    return text

In [ ]:
def clean_text(text: str) -> str:
    """Clean and normalize text."""
    text = re.sub(r"\s+", " ", text)
    return text.strip()

In [ ]:
# -------------------------------
# STEP 3: Load all PDFs
# -------------------------------
pdf_files = {
    "FAQ": "/content/FoodDeliveryApp_FAQs (1).pdf",
    "SLA": "/content/SwiftBite_SLA_Document.pdf",
    "POLICY": "/content/SwiftBite_Internal_Policy.pdf",
    "ESCALATION": "/content/Food Delivery App - Escalation Workflows.pdf"
}

def load_documents(pdf_dict: Dict[str, str]) -> List[Dict]:
    """Load multiple PDFs into a list of dicts with metadata."""
    documents = []
    for doc_type, path in pdf_dict.items():
        text = load_pdf(path)
        text = clean_text(text)
        documents.append({
            "page_content": text,
            "metadata": {
                "document_type": doc_type,
                "source": os.path.basename(path)
            }
        })
    return documents

documents = load_documents(pdf_files)
print(f"Loaded {len(documents)} documents")

Loaded 4 documents


In [ ]:
# -------------------------------
# STEP 4: Chunk Documents
# -------------------------------

def chunk_faq(text: str):
    blocks = re.split(r"(?=Q:\s)", text)
    chunks = []

    for b in blocks:
        if "A:" in b:
            question = b.split("Q:", 1)[1].split("A:", 1)[0].strip()
            chunks.append({
                "content": b.strip(),
                "question": question
            })

    return chunks


In [ ]:
def chunk_numbered_sections(text: str):
    text = text.replace("\r", " ").strip()

    sections = re.split(
        r"(?=\b\d+\.\d+\s+[A-Z])",
        text
    )

    return [
        s.strip()
        for s in sections
        if len(s.strip()) > 120
    ]


In [ ]:
from langchain_core.documents import Document

def chunk_by_structure(doc):
    text = doc["page_content"]
    doc_type = doc["metadata"]["document_type"]

    if doc_type == "FAQ":
        return chunk_faq(text)

    return chunk_numbered_sections(text)


In [ ]:
from langchain_core.documents import Document

final_documents = []

for doc in documents:
    chunks = chunk_by_structure(doc)

    for i, chunk in enumerate(chunks):

        metadata = {
            **doc["metadata"],
            "chunk_id": f"{doc['metadata']['document_type']}_{i}",
            "chunk_type": "faq" if doc["metadata"]["document_type"] == "FAQ" else "section"
        }

        # ✅ FAQ chunks now return dicts: {"content": ..., "question": ...}
        if doc["metadata"]["document_type"] == "FAQ":
            page_content = chunk["content"]
            metadata["faq_question"] = chunk["question"]
        else:
            # Non-FAQ chunks are plain strings
            page_content = chunk

        final_documents.append(
            Document(
                page_content=page_content,  # ✅ ALWAYS a string
                metadata=metadata
            )
        )

print(f"Total structured chunks: {len(final_documents)}")
print(final_documents[0])


Total structured chunks: 170
page_content='Q: How do I create an account? A: Download the app from the App Store or Google Play. Open the app and select 'Sign Up'. You can register using your email address, phone number, or by linking your Google/Facebook account.' metadata={'document_type': 'FAQ', 'source': 'FoodDeliveryApp_FAQs (1).pdf', 'chunk_id': 'FAQ_0', 'chunk_type': 'faq', 'faq_question': 'How do I create an account?'}


In [ ]:
# -------------------------------
# STEP 5: Create embeddings
# -------------------------------
embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)


/tmp/ipython-input-789377893.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# -------------------------------
# STEP 6: Initialize Chroma vector store
# -------------------------------
persist_dir = "/content/chroma_food_delivery"

vectorstore = Chroma.from_documents(
    documents=final_documents,
    embedding=embedding_model,
    persist_directory=persist_dir
)

vectorstore.persist()
print("Vector store persisted successfully.")

Vector store persisted successfully.


/tmp/ipython-input-1471848135.py:12: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstore.persist()


In [ ]:
print("Total chunks embedded:", vectorstore._collection.count())


Total chunks embedded: 170


In [ ]:
results = vectorstore.similarity_search(
    "How can I search, customize, and schedule food orders?",
    k=3
    )
print(results)

[Document(metadata={'document_type': 'FAQ', 'faq_question': 'How do I search for specific food?', 'source': 'FoodDeliveryApp_FAQs (1).pdf', 'chunk_id': 'FAQ_5', 'chunk_type': 'faq'}, page_content="Q: How do I search for specific food? A: Use the search bar at the top of the home screen. You can search by restaurant name, dish name, or cuisine type (e.g., 'Pizza', 'Vegan', 'Chinese')."), Document(metadata={'source': 'FoodDeliveryApp_FAQs (1).pdf', 'faq_question': 'Can I customize my order?', 'chunk_type': 'faq', 'document_type': 'FAQ', 'chunk_id': 'FAQ_6'}, page_content="Q: Can I customize my order? A: Absolutely. When selecting a dish, look for the 'Add-ons' or 'Customize' section. You can remove ingredients, choose spice levels, or add extra toppings."), Document(metadata={'chunk_id': 'FAQ_14', 'source': 'FoodDeliveryApp_FAQs (1).pdf', 'document_type': 'FAQ', 'chunk_type': 'faq', 'faq_question': 'Can I get an invoice for my order?'}, page_content="Q: Can I get an invoice for my order?

**Task 2**

In [ ]:
from typing import Optional, Dict, Any, List, Tuple
from langchain_core.documents import Document

# --------------------------------------------------
# Core Retrieval Function (with optional filtering)
# --------------------------------------------------

def retrieve_with_scores(
    query: str,
    vectorstore,
    doc_type: Optional[str] = None,
    k: int = 5
) -> List[Tuple[Document, float]]:
    """
    Retrieve top-k documents with distance scores.
    Lower score = higher semantic similarity (Chroma distance).
    """

    if doc_type:
        results = vectorstore.similarity_search_with_score(
            query=query,
            k=k,
            filter={"document_type": doc_type}
        )
        retrieval_mode = "FILTERED"
    else:
        results = vectorstore.similarity_search_with_score(
            query=query,
            k=k
        )
        retrieval_mode = "GLOBAL"

    return results, retrieval_mode


In [ ]:
# --------------------------------------------------
# Retrieval with Relevance Guardrail
# --------------------------------------------------

def retrieve_with_relevance_guardrail(
    query: str,
    vectorstore,
    doc_type: Optional[str] = None,
    k: int = 5,
    score_threshold: float = 1.5,
    min_docs: int = 1
) -> Dict[str, Any]:
    """
    Applies strict relevance filtering and returns
    a structured retrieval response for downstream LangGraph nodes.
    """

    results, retrieval_mode = retrieve_with_scores(
        query=query,
        vectorstore=vectorstore,
        doc_type=doc_type,
        k=k
    )

    # Chroma returns distance scores (lower = better)
    relevant_docs = [
        (doc, score)
        for doc, score in results
        if score <= score_threshold
    ]

    # Sort by best relevance
    relevant_docs = sorted(relevant_docs, key=lambda x: x[1])

    # Guardrail: No or insufficient relevant documents
    if len(relevant_docs) < min_docs:
        return {
            "status": "NO_RELEVANT_DOCS",
            "message": "The requested information is not available in the internal documentation.",
            "retrieval_mode": retrieval_mode,
            "query": query,
            "results": []
        }

    return {
        "status": "SUCCESS",
        "message": "Relevant documents retrieved successfully.",
        "retrieval_mode": retrieval_mode,
        "query": query,
        "results": relevant_docs
    }

In [ ]:
query = "how to creat account"

response = retrieve_with_relevance_guardrail(
    query=query,
    vectorstore=vectorstore
)

print(response["status"])
print(response["message"])
print(response["results"])


SUCCESS
Relevant documents retrieved successfully.
[(Document(metadata={'chunk_type': 'faq', 'source': 'FoodDeliveryApp_FAQs (1).pdf', 'chunk_id': 'FAQ_0', 'document_type': 'FAQ', 'faq_question': 'How do I create an account?'}, page_content="Q: How do I create an account? A: Download the app from the App Store or Google Play. Open the app and select 'Sign Up'. You can register using your email address, phone number, or by linking your Google/Facebook account."), 0.9340939521789551)]


**Task 3**

In [ ]:
import json
from pathlib import Path

LOG_FILE = Path("audit_logs.jsonl")

In [ ]:
from dataclasses import dataclass, field
from typing import List, Optional
from langchain_core.documents import Document
from datetime import datetime, timezone
import uuid

@dataclass
class GraphState:
    # Core
    query: str

    # Session
    session_id: str = field(default_factory=lambda: str(uuid.uuid4()))
    timestamp: str = field(default_factory=lambda: datetime.now(timezone.utc).isoformat())

    # Reasoning
    intent: Optional[str] = None
    retrieved_docs: List[Document] = field(default_factory=list)
    grounded: bool = False
    safe: bool = True

    # Output
    answer: Optional[str] = None

In [ ]:
import re

def redact_pii(text: str) -> str:
    text = re.sub(r'\b\d{10}\b', '[REDACTED_PHONE]', text)
    text = re.sub(r'[\w\.-]+@[\w\.-]+\.\w+', '[REDACTED_EMAIL]', text)
    text = re.sub(r'\b\d{4}-\d{4}-\d{4}-\d{4}\b', '[REDACTED_CARD]', text)
    return text


In [ ]:
from sentence_transformers import util

def select_relevant_faqs(query, faq_docs, embedding_model, score_threshold=0.5):
    query_emb = embedding_model.embed_query(query)

    scored = []
    for doc in faq_docs:
        faq_q = doc.metadata.get("faq_question")
        if not faq_q:
            continue

        faq_emb = embedding_model.embed_query(faq_q)
        score = util.cos_sim(query_emb, faq_emb).item()
        scored.append((score, doc))

    if not scored:
        return [], 0.0

    scored.sort(reverse=True, key=lambda x: x[0])

    top_score = scored[0][0]

    selected = [scored[0][1]]

    if len(scored) > 1 and scored[1][0] >= score_threshold:
        selected.append(scored[1][1])

    return selected, top_score


In [ ]:
def intent_detection_node(state: GraphState) -> GraphState:
    meta_keywords = ["what policies are available", "what documents", "what is available", "list", "overview"]

    if any(k in state.query.lower() for k in meta_keywords):
        state.intent = "OVERVIEW"
    else:
        state.intent = "FACT_LOOKUP"

    return state


In [ ]:
def retrieval_node(state: GraphState) -> GraphState:
    if state.intent == "OVERVIEW":
        state.retrieved_docs = []
        return state

    # 🔑 HIGH-RECALL retrieval (do NOT filter yet)
    retrieved = vectorstore.similarity_search(
        state.query,
        k=8  # higher recall so correct FAQ is included
    )

    if not retrieved:
        state.retrieved_docs = []
        state.grounded = False
        state.answer = "I could not find relevant information in the internal documents."
        return state

    state.retrieved_docs = retrieved
    return state


In [ ]:
def grounding_node(state: GraphState) -> GraphState:

    if state.intent == "OVERVIEW":
        state.grounded = True
        return state

    if not state.retrieved_docs:
        state.grounded = False
        state.answer = "I could not find relevant information in the internal documents."
        return state

    query_terms = set(state.query.lower().split())
    grounded_docs = []

    for doc in state.retrieved_docs:
        doc_terms = set(doc.page_content.lower().split())
        if query_terms & doc_terms:
            grounded_docs.append(doc)

    if not grounded_docs:
        state.grounded = False
        state.answer = "I could not find relevant information in the internal documents."
        return state

    state.retrieved_docs = grounded_docs
    state.grounded = True
    return state


In [ ]:
def safety_guardrail_node(state: GraphState) -> GraphState:

    if state.intent == "OVERVIEW":
        state.safe = True
        return state

    if not state.grounded:
        state.safe = False
        return state

    sanitized_docs = []
    for doc in state.retrieved_docs:
        sanitized_docs.append(
            Document(
                page_content=redact_pii(doc.page_content),
                metadata=doc.metadata
            )
        )

    state.retrieved_docs = sanitized_docs
    state.safe = True
    return state

In [ ]:
def response_node(state: GraphState) -> GraphState:
    # 1. Handle overview intent (system-generated)
    if state.intent == "OVERVIEW":
        state.answer = (
            "[System Overview]\n"
            "The following internal documents are available:\n"
            "- FAQs\n"
            "- Service Level Agreements (SLA)\n"
            "- Internal Policies\n"
            "- Escalation Workflows\n\n"
            "Please ask a specific question about any of these documents."
        )
        return state

    # 2. Enforce grounding & safety
    if not state.grounded or not state.safe:
        state.answer = "I could not find relevant information in the internal documents."
        return state

    # 3. Enforce retrieval
    if not state.retrieved_docs:
        state.answer = "I could not find relevant information in the internal documents."
        return state

    # 4. Select primary and supporting documents (fallback only)
    primary_doc = state.retrieved_docs[0]
    supporting_docs = state.retrieved_docs[1:3]
    primary_content = primary_doc.page_content.strip()

    # 5. FAQ-style extraction (semantic + confidence-based + out-of-scope guardrail)
    faq_docs = [
        d for d in state.retrieved_docs
        if d.metadata.get("document_type") == "FAQ"
    ]

    if faq_docs:
        selected_faqs, top_score = select_relevant_faqs(
            state.query,
            faq_docs,
            embedding_model
        )

        # OUT-OF-SCOPE GUARDRAIL
        if top_score < 0.45:
            state.answer = "I could not find relevant information in the internal documents."
            return state

        faq_answers = [
            doc.page_content.split("A:", 1)[1].strip()
            for doc in selected_faqs
            if "A:" in doc.page_content
        ]

        if faq_answers:
            final_answer = "\n\n".join(faq_answers)

            state.answer = (
                f"{final_answer}\n\n"
                f"Primary Source: {selected_faqs[0].metadata.get('source')} (FAQ)"
            )
            return state

    # 6. Fallback for SLA / Policy / Workflow chunks
    state.answer = (
        f"{primary_content}\n\n"
        f"Primary Source: {primary_doc.metadata.get('source')} "
        f"({primary_doc.metadata.get('document_type')})"
    )

    if supporting_docs:
        state.answer += "\nSupporting Sources: " + ", ".join(
            f"{doc.metadata.get('source')} ({doc.metadata.get('document_type')})"
            for doc in supporting_docs
        )

    return state


In [ ]:
def log_interaction(state: GraphState):
    log_entry = {
        # Session metadata
        "session_id": state.session_id,
        "timestamp": state.timestamp,

        # User input
        "query": state.query,
        "intent": state.intent,

        # System decision flags
        "grounded": state.grounded,
        "safe": state.safe,
        "status": (
            "SUCCESS" if state.grounded and state.safe
            else "NO_RELEVANT_DOCS"
        ),

        # Final output
        "answer": state.answer,

        # Primary grounding reference (important for audits)
        "primary_chunk_id": (
            state.retrieved_docs[0].metadata.get("chunk_id")
            if state.retrieved_docs else None
        ),

        # Retrieved context (redacted + traceable)
        "retrieved_documents": [
            {
                "source": doc.metadata.get("source"),
                "document_type": doc.metadata.get("document_type"),
                "chunk_id": doc.metadata.get("chunk_id"),
                "content_preview": doc.page_content[:200].replace("\n", " ")
            }
            for doc in state.retrieved_docs
        ]
    }

    with LOG_FILE.open("a") as f:
        f.write(json.dumps(log_entry) + "\n")


In [ ]:
def audit_log_node(state: GraphState) -> GraphState:
    log_interaction(state)
    return state

In [ ]:
from langgraph.graph import StateGraph

graph = StateGraph(GraphState)

graph.add_node("intent", intent_detection_node)
graph.add_node("retrieval", retrieval_node)
graph.add_node("grounding", grounding_node)
graph.add_node("safety", safety_guardrail_node)
graph.add_node("response", response_node)
graph.add_node("audit", audit_log_node)

graph.set_entry_point("intent")

graph.add_edge("intent", "retrieval")
graph.add_edge("retrieval", "grounding")
graph.add_edge("grounding", "safety")
graph.add_edge("safety", "response")
graph.add_edge("response", "audit")   # 👈 Task 5

app = graph.compile()


In [ ]:
final_state = app.invoke(
    GraphState(query="How do I delete account?")
)

print(final_state["answer"])

We're sad to see you go. Navigate to Profile > Settings > Privacy > Delete Account. Note that this action is irreversible and you will lose all loyalty points. Page 1 FoodDeliveryApp - Support Knowledge Base 2. Placing an Order

Download the app from the App Store or Google Play. Open the app and select 'Sign Up'. You can register using your email address, phone number, or by linking your Google/Facebook account.

Primary Source: FoodDeliveryApp_FAQs (1).pdf (FAQ)


**Task - 4**

In [ ]:
# ✅ Task 4 validation tests

queries = [
    "Can I order from multiple restaurants at once?",
    "What policies are available?",
    "How to apply for a home loan?"
]

for q in queries:
    result = app.invoke({"query": q})
    print("\nQuery:", q)
    print("Answer:", result["answer"])



Query: Can I order from multiple restaurants at once?
Answer: Currently, each order must be from a single restaurant to ensure delivery efficiency. To order from two places, please place two separate orders. Page 2 FoodDeliveryApp - Support Knowledge Base 3. Payments & Billing

Absolutely. When selecting a dish, look for the 'Add-ons' or 'Customize' section. You can remove ingredients, choose spice levels, or add extra toppings.

Primary Source: FoodDeliveryApp_FAQs (1).pdf (FAQ)

Query: What policies are available?
Answer: [System Overview]
The following internal documents are available:
- FAQs
- Service Level Agreements (SLA)
- Internal Policies
- Escalation Workflows

Please ask a specific question about any of these documents.

Query: How to apply for a home loan?
Answer: I could not find relevant information in the internal documents.


**Task - 5**

In [ ]:
import json

def view_audit_logs(path="audit_logs.jsonl", limit=5):
    print("\n===== AUDIT LOGS =====\n")
    with open(path) as f:
        for line in list(f)[-limit:]:
            print(json.dumps(json.loads(line), indent=2))
            print("-" * 40)


In [ ]:
view_audit_logs(limit=1)



===== AUDIT LOGS =====

{
  "session_id": "78503384-a22b-41cb-80f2-fe3310487a56",
  "timestamp": "2025-12-30T16:11:40.475033+00:00",
  "query": "How to apply for a home loan?",
  "intent": "FACT_LOOKUP",
  "grounded": true,
  "safe": true,
  "status": "SUCCESS",
  "answer": "I could not find relevant information in the internal documents.",
  "primary_chunk_id": "FAQ_12",
  "retrieved_documents": [
    {
      "source": "FoodDeliveryApp_FAQs (1).pdf",
      "document_type": "FAQ",
      "chunk_id": "FAQ_12",
      "content_preview": "Q: How do I add a promo code? A: On the checkout screen, tap 'Apply Coupon'. Enter the code and tap 'Apply'. The discount will be reflected in the total immediately."
    },
    {
      "source": "SwiftBite_Internal_Policy.pdf",
      "document_type": "POLICY",
      "chunk_id": "POLICY_31",
      "content_preview": "6.1 Merchant Onboarding & Validation To partner with SwiftBite, restaurants must provide valid business licenses, health inspection certific

**Task 6**

In [ ]:
def run_customer_support_query(query: str):
    state = GraphState(query=query)
    final_state = app.invoke(state)
    return final_state

In [ ]:
run_customer_support_query("How do I create and manage my FoodDeliveryApp account?")

{'query': 'How do I create and manage my FoodDeliveryApp account?',
 'session_id': 'cea53f13-e186-4fff-894e-cf307b46f45c',
 'timestamp': '2025-12-30T16:11:51.342268+00:00',
 'intent': 'FACT_LOOKUP',
 'retrieved_docs': [Document(metadata={'document_type': 'FAQ', 'faq_question': 'How do I delete my account?', 'source': 'FoodDeliveryApp_FAQs (1).pdf', 'chunk_type': 'faq', 'chunk_id': 'FAQ_4'}, page_content="Q: How do I delete my account? A: We're sad to see you go. Navigate to Profile > Settings > Privacy > Delete Account. Note that this action is irreversible and you will lose all loyalty points. Page 1 FoodDeliveryApp - Support Knowledge Base 2. Placing an Order"),
  Document(metadata={'document_type': 'FAQ', 'faq_question': "I see a 'Network Error' message.", 'chunk_type': 'faq', 'chunk_id': 'FAQ_44', 'source': 'FoodDeliveryApp_FAQs (1).pdf'}, page_content="Q: I see a 'Network Error' message. A: Check your internet connection. Switch between Wi-Fi and mobile data. If the server is down

In [ ]:
test_queries = [
    # FAQ
    "Can I order from multiple restaurants at once?",

    # SLA
    "In the event of a security breach involving customer PII, how many hours does SwiftBite have to officially notify us?",

    # Policy
    "How many days of Paid Time Off (PTO) and Sick Leave am I entitled to per year?",

    # ESCALATION
    "What core principles and mandatory rules govern escalations?",

    # Out-of-scope (hallucination test)
    "How to apply for a home loan?",

    # Overview / meta
    "What policies are available?"
]

for q in test_queries:
    result = run_customer_support_query(q)
    print("\nQUERY:", q)
    print("ANSWER:", result["answer"])



QUERY: Can I order from multiple restaurants at once?
ANSWER: Currently, each order must be from a single restaurant to ensure delivery efficiency. To order from two places, please place two separate orders. Page 2 FoodDeliveryApp - Support Knowledge Base 3. Payments & Billing

Absolutely. When selecting a dish, look for the 'Add-ons' or 'Customize' section. You can remove ingredients, choose spice levels, or add extra toppings.

Primary Source: FoodDeliveryApp_FAQs (1).pdf (FAQ)

QUERY: In the event of a security breach involving customer PII, how many hours does SwiftBite have to officially notify us?
ANSWER: 6.2 Data Breach Notification In the event of a confirmed data breach involving Personally Identifiable Information (PII), SwiftBite guarantees notification to the Customer within 72 hours of discovery. This notification will include the nature of the breach, the data affected, and the mitigation steps taken.

Primary Source: SwiftBite_SLA_Document.pdf (SLA)
Supporting Sources: 

**Gradio**

In [ ]:
import gradio as gr

# ----------------------------------------
# Gradio Chat Function
# ----------------------------------------
def chat(query):
    final_state = app.invoke(
        GraphState(query=query)
    )

    # Handle BOTH dict and GraphState outputs safely
    if isinstance(final_state, dict):
        answer = final_state.get("answer", "No answer generated.")
    else:
        answer = getattr(final_state, "answer", None) or "No answer generated."

    answer_clean = answer.strip().lower()

    #OUT-OF-SCOPE → NO SOURCES
    if answer_clean.startswith("i could not find relevant information"):
        sources_text = "No sources"
        return answer, sources_text

    # ✅ IN-SCOPE → extract ONLY cited Primary Source from answer
    sources = []
    if "Primary Source:" in answer:
        source_line = answer.split("Primary Source:", 1)[1].strip()
        sources.append(f"- {source_line}")

    sources_text = "\n".join(sources) if sources else "No sources"

    return answer, sources_text


# ----------------------------------------
# Gradio UI Layout
# ----------------------------------------
with gr.Blocks(title="Customer Support Intelligence System") as demo:
    gr.Markdown("## 📞 Customer Support Intelligence System")
    gr.Markdown(
        "Ask questions about **FAQs, SLAs, Internal Policies, and Escalation Workflows**.\n"
        "Responses are grounded strictly in internal documents."
    )

    query_input = gr.Textbox(
        label="Customer Question",
        placeholder="e.g., Can I order from multiple restaurants at once?"
    )

    answer_output = gr.Textbox(
        label="Answer",
        lines=6
    )

    source_output = gr.Textbox(
        label="Sources",
        lines=4
    )

    submit_btn = gr.Button("Ask")

    submit_btn.click(
        fn=chat,
        inputs=query_input,
        outputs=[answer_output, source_output]
    )

demo.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://4b3a626971f88ea015.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
